In [175]:
import numpy as np
from cvxopt import matrix, solvers, spmatrix, sparse
import random
from cvxopt import normal
import numpy as np


In [176]:
#Implement AGKM
# -*- coding: utf-8 -*-

from cvxopt import matrix, solvers, spmatrix, sparse

#============================================================
# distFromCon: L1 distance between a vector and a convex hull
#------------------------------------------------------------
# params : 
#    V vector 
#    P matrix which rows define the convex hull
#============================================================

def distFromConv(V, P):
    # Make sure that the parameters are supported by cvxopt
    V = matrix(V)
    P = matrix(P).trans()
    # Dimension of the vector space
    m = len(V)
    # Number of lements defining the hull
    n = P.size[1]
    # Vectors vith only ones as entry
    onesn = matrix(1.0,(n,1))
    onesm = matrix(1.0,(m,1))
    # Identity matrices
    Idm = spmatrix(1.0, range(m),range(m))
    Idn = spmatrix(1.0, range(n),range(n))
    # Zero matrix
    Znm = spmatrix(0,[n-1],[m-1])
    Zn = spmatrix(0,[n-1],[0])
    Zm = spmatrix(0,[m-1],[0])
    # Expressing the problem as an LP problem
    # Inequality constraints
    A = sparse([ [P,-P,-Idn], [-Idm,-Idm,Znm] ]) 
    C = matrix([[V,-V,Zn]])
    e = matrix([ [Zn, onesm] ])
    # Equality constraint
    G = matrix([onesn,Zm]).trans()
    # Computing the solution
    solution = solvers.lp(e,A,C,G,matrix(1.0))['x']
    # Computing the L1-closest point in the hull
    u = solution[range(n)]
    V_hull = P*u
    return sum(abs(V-V_hull))


#============================================================
# Norm_inf1: solve the optimization problem argmin ||X-FW||_{\infty,1} with X, W known
#------------------------------------------------------------
# params : 
#    X matrix of shape f * n
#    W matrix of shape r * n
#============================================================


def Norm_inf1 (X,W):
    # X, W are two matrix of shape respectively f*n et r*n
    f,n = X.size
    r = W.size[0]
    
    print f,n,r
    
    P = matrix(W).trans()
    F = matrix(1.0,(f,r))
    
    onesn = matrix(1.0,(n,1))
    Idn = spmatrix(1.0, range(n),range(n))
    Idr = spmatrix(1.0, range(r),range(r))
    Zrn = spmatrix(0,[r-1],[n-1])
    Zr = spmatrix(0,[r-1],[0])
    #Zn = spmatrix(0,[n-1],[0])
    A = sparse([ [P,-P,-Idr], [-Idn,-Idn,Zrn] ]) 
    
    for i in range(f):
        V =  X[i,:].trans()
        C = matrix([[V,-V,Zr]])
        e = matrix([ [Zr, onesn] ])
        solution = solvers.lp(e,A,C)['x']
        F[i,:] = solution[range(r)].trans()
    return F    




#============================================================
# AGKM: Approximably Separable non-negative matrix 
# factorization
#------------------------------------------------------------
# params : 
#    X matrix to be factorized 
#    alpha simplicial robustness if hott topics (see paper)
#    epsilon  approximation control parameter (see paper)
#============================================================

def AGKM(X, alpha, epsilon):
    # Make sure that X is supported by cvxopt library
    X = matrix(X)
    # Number of features
    nrows = X.size[0]
    # Pairwise L1 distances between rows 
    D = matrix(0.0, (nrows,nrows))
    for i in range(nrows):
        for j in range(i+1,nrows):
            D[i,j] = sum(abs(X[i,:]-X[j,:]))
            D[j,i] = D[i,j]
    # Finding the set R of rows that are simplicial
    R = []
    # Computing minimal distance between elements of R
    minDist = 5*epsilon/alpha+2*epsilon
    for k in range(nrows):    
        # Set of indices of rows far enough
        Nk = []
        for j in range(nrows):
            if (D[k,j] >= minDist and k!=j): Nk = Nk+[j]
        # Matrix which rows are the ones indexed by Nk
        P = X[Nk,:]
        # Computing the L1 distance between Xk and the convex 
        # hull defined by the rows of Nk
        deltak = distFromConv(X[k,:].trans(),P)
        # Add the k-th row to R if deltak is large enough
        if deltak > 2*epsilon: R = R+[k]
    # Cluster the elements of R ot obtain the rows of 
    # W (due to the particular geometry of the problem,
    # the clustering is done in O(n), see paper)
    # L1 diameter of the clusters
    diam = 10*epsilon/alpha+epsilon
    clusterR = []
    for i in R:
        if filter(lambda x: x<= diam, D[i,clusterR]) == []:
            clusterR = clusterR+[i]
    # W Matrix of canonical rows
    W = X[clusterR,:]
    
    F = Norm_inf1(X,W)
    return (F,W)


In [201]:
#============================================================
# permutation:take the i-th element and put it on the first position
#             then sort the rest in descending order
#------------------------------------------------------------
# params : 
#    X : vector array type
#    i: an index
#return : 
#    Z : a sorted vector
#    index
#    C_ii : i-th element
#============================================================

def permutation (X,i):# return a column Z verified the hypothesis z_2>...z_f
    #X is a vector, i-th column of the Matrix C
    #
    C_ii=X[i]
    a=np.max(X)+1
    X[i]=a
    index=np.argsort(X)[::-1]
    Z=np.sort(X)[::-1]
    Z[0]=C_ii
    return Z, index, C_ii
    

#============================================================
# proj_01:projection on [0,1] of a number
#------------------------------------------------------------
# params : 
#    x : a number
#============================================================    
    
def proj_01(x): #projection on interval [0,1]
    if x>1 :return 1
    if x<0 :return 0
    else :  return x
    

#============================================================
# proj_on_phi_0:implementation of Column Squishing
#          (projection of i-th column implicitly )
#------------------------------------------------------------
# params : 
#    Z , index given by the function permutation 
#============================================================        
    
def proj_on_phi_0 (Z,index):#algo 5 
    f=Z.shape[0] 
    x=np.zeros(f)
    xx = np.zeros(f)
    mu = Z[0]
    for k in range(1,f): 
        if Z[k] <= proj_01(mu):
            k_c=k
            break
        else:
            mu= k/(k+1)*mu+1./(k+1)*Z[k]
    k_c=f    
    print 'k_c=',k_c,'mu=',mu
    x[0]=proj_01(mu)
    for k in range(1,k_c):
        x[k]=proj_01(mu)
    for k in range(k_c,f): 
        x[k]=max(Z[k],0)
    #print 'x=',x    
    #to return X, after permutation inverse
    for i in range(f):
        xx[index[i]]=x[i]
    return xx

#============================================================
# proj_on_phi_final: Complete projection of a matrix on Phi_0
#------------------------------------------------------------
# params : 
#     C : a matrix  
#============================================================  

def proj_on_phi_final(C):
    f = C.shape[0]
    D = np.zeros((f,f))
    for i in range(f):
        X = np.squeeze(np.asarray(C[:,i]))
        print X.shape
        Z, index, C_ii=permutation(X,i)
        #there always has a problem of compatibility between matrix, array of i-th column of C
        #D[:,i]=np.matrix(proj_on_phi_0(Z,index)).T
        D[:,i]=proj_on_phi_0(Z,index)

    return D



#============================================================
# proj2: projection directe of a matrix in Phi_0 by projecting
#          its diagonal coeff in [0,1] and keep its positive coeff
#------------------------------------------------------------
# params : 
#    C: a matrix
#============================================================  
def proj2(C):
    f = C.shape[0]
    for i in range(f):
        for j in range(f):
            if i==j:
                C[i,j]=proj_01(C[i,j])
            else:
                C[i,j]=max(0,C[i,j])
    return 


        
  
        
#============================================================
# Hottpixx: implementation of Hottopixx
#           a NNF of a matrix X
#------------------------------------------------------------
# params : 
#    X: a matrix 
#    N: iteration number
#============================================================ 
    
    
def Hottpixx(X,s_p,s_d,N):
    f,n = X.shape
    p = np.arange(f)/(1000.)
    C = np.zeros((f,f))
    beta = 0
    w = np.ones(f)
    GDindex=np.arange(n)
    mu = np.zeros(f)
    for j in range(f):#μj is the number of nonzeros in row j divided by n.
        #mu[j] = 1/(np.sum(X[j,:] != 0)+0.0)
        mu[j] = 1./n
    for t in range(N):
        for i in range(n):
            k=random.choice(GDindex)
            C = C + s_p * (np.matrix(np.sign(X[:,k]-np.dot(C,X[:,k]))).dot(np.matrix(X[:,k]).T)) - s_p*np.diag(mu*(beta-p))  
            #w[i] = w[i] + s_d * (np.sum(np.abs(X[i,:]-np.dot(C,X)[i,:]))- 0.1)
            #print 'shape=',(np.matrix(np.sign(X[:,k]-np.dot(C,X[:,k]))).dot(np.matrix(X[:,k]).T))
            #print 'C=',C
        #print 'C.shape=',C.shape
        
        C = proj_on_phi_final(C)
        #proj2(C)
        #print 'C=',C
        
        beta = beta + s_d*(np.trace(C)-r)#we assume r is known, r is a global variable
        print beta
        
        print t,'-th iteration is done'
    #print C
    diag=np.diag(C)
    print 'diag=',diag
    index=np.argsort(diag)[::-1]
    I=index[:r]
    #print I
    W = X[I]
    
    #convert X,W into matrix type defined in cvxopt in order to apply the argmin function
    W1=matrix(W)
    X1=matrix(X)
    F = Norm_inf1(X1,W1)
    #convert again F into numpy matrix
    F1=np.array(F)
    return F1,W



In [202]:
#Generate a matrix X sperable by creating artificially two matrices F and W, and set X= FW
W1=np.matrix([[1,0,0.,0,0,0],[0,1,0,0,0,0]])
F1=np.matrix([[0.25,0.75],[1,0],[.3,.7],[.5,.5],[.9,.1],[0,1]])
Y = np.dot(F1,W1)
#bruit = 0.001 *np.random.randn(6, 6)
#Z= Y+ bruit
X = matrix(Y)#convert to matrix type of cvxopt library in order to apply AGKM function

In [203]:
# set r=2 
r=2
A,B=Hottpixx(Y,0.1,0.1,10)

#the key consists of the choice of W, so we check wether C is correct.
print 'W=',B
# and then check the difference with the norm l1 of X et F'W'
print 'the difference is: ',np.sum(np.abs(A.dot(B)-Y))

(6,)
k_c= 6 mu= 0.025
(6,)
k_c= 6 mu= 0.1001
(6,)
k_c= 6 mu= 0.0302
(6,)
k_c= 6 mu= 0.0503
(6,)
k_c= 6 mu= 0.0904
(6,)
k_c= 6 mu= 0.0005
-0.17035
0 -th iteration is done
(6,)
k_c= 6 mu= 0.067035
(6,)
k_c= 6 mu= 0.217235
(6,)
k_c= 6 mu= 0.077435
(6,)
k_c= 6 mu= 0.117635
(6,)
k_c= 6 mu= 0.197835
(6,)
k_c= 6 mu= 0.018035
-0.300829
1 -th iteration is done
(6,)
k_c= 6 mu= 0.1721179
(6,)
k_c= 6 mu= 0.2474179
(6,)
k_c= 6 mu= 0.1777179
(6,)
k_c= 6 mu= 0.1980179
(6,)
k_c= 6 mu= 0.2183179
(6,)
k_c= 6 mu= 0.1486179
-0.38460826
2 -th iteration is done
(6,)
k_c= 6 mu= 0.210578726
(6,)
k_c= 6 mu= 0.285978726
(6,)
k_c= 6 mu= 0.216378726
(6,)
k_c= 6 mu= 0.236778726
(6,)
k_c= 6 mu= 0.257178726
(6,)
k_c= 6 mu= 0.187578726
-0.4451610244
3 -th iteration is done
(6,)
k_c= 6 mu= 0.0309297876667
(6,)
k_c= 6 mu= 0.43059482844
(6,)
k_c= 6 mu= 0.031063121
(6,)
k_c= 6 mu= 0.0311297876667
(6,)
k_c= 6 mu= 0.39209482844
(6,)
k_c= 6 mu= 0.23259482844
-0.530320306235
4 -th iteration is done
(6,)
k_c= 6 mu= 0.23396181

In [207]:
F,W = AGKM(X,0.5,0.0001)

print F, W
#convert F, W into array type and calculate the difference
F = np.asarray(F)
W = np.asarray(W)
X = np.asarray(X)
print 'the difference is: ',np.sum(np.abs(F.dot(W)-X))

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+01  4e+00  2e+00  1e+00
 1: -5.9729e-01  1.4420e-02  4e-01  5e-01  3e-01  7e-01
 2: -4.1256e-02 -2.0100e-02  4e-02  3e-02  2e-02  3e-02
 3: -4.0991e-04 -2.1002e-04  4e-04  3e-04  2e-04  3e-04
 4: -4.0975e-06 -2.0994e-06  4e-06  3e-06  2e-06  3e-06
 5: -4.0975e-08 -2.0994e-08  4e-08  3e-08  2e-08  3e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+01  4e+00  2e+00  1e+00
 1: -3.9344e-01  2.3886e-01  1e+00  6e-01  4e-01  8e-01
 2:  6.1873e-04  1.0867e-01  2e-01  8e-02  5e-02  1e-01
 3:  1.6997e-01  2.1123e-01  1e-01  4e-02  2e-02  5e-02
 4:  1.9970e-01  2.0010e-01  1e-03  4e-04  3e-04  5e-04
 5:  2.0000e-01  2.0000e-01  1e-05  4e-06  3e-06  5e-06
 6:  2.0000e-01  2.0000e-01  1e-07  4e-08  3e-08  5e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+01  4e+00  2e+00  1e+00
 1: -6

In [208]:
#even wokrs with a small noise
W2=np.matrix([[1,0,0.,0,0,0],[0,1,0,0,0,0]])
F2=np.matrix([[0.25,0.75],[1,0],[.3,.7],[.5,.5],[.9,.1],[0,1]])
Y = np.dot(F1,W1)
bruit = 0.001 *np.random.randn(6, 6)
Y= Y+ bruit
X = matrix(Y)#convert to matrix type of cvxopt library in order to apply AGKM function

In [209]:
# set r=2 
r=2
A,B=Hottpixx(Y,0.1,0.1,10)

#the key consists of the choice of W, so we check wether C is correct.
print 'W=',B
# and then check the difference with the norm l1 of X et F'W'
print 'the difference is: ',np.sum(np.abs(A.dot(B)-Y))

(6,)
k_c= 6 mu= 0.174821595204
(6,)
k_c= 6 mu= 0.0166519712454
(6,)
k_c= 6 mu= 0.170533296146
(6,)
k_c= 6 mu= 0.15059131616
(6,)
k_c= 6 mu= 0.0150159633543
(6,)
k_c= 6 mu= 0.200631574448
-0.127175428344
0 -th iteration is done
(6,)
k_c= 6 mu= 0.21292954444
(6,)
k_c= 6 mu= 0.129882137761
(6,)
k_c= 6 mu= 0.213642454114
(6,)
k_c= 6 mu= 0.213987025652
(6,)
k_c= 6 mu= 0.118517356858
(6,)
k_c= 6 mu= 0.214414472019
-0.21683812926
1 -th iteration is done
(6,)
k_c= 6 mu= 0.309624221631
(6,)
k_c= 6 mu= 0.152177402117
(6,)
k_c= 6 mu= 0.305518767204
(6,)
k_c= 6 mu= 0.027301527401
(6,)
k_c= 6 mu= 0.13077713863
(6,)
k_c= 6 mu= 0.337103847608
-0.290587838801
2 -th iteration is done
(6,)
k_c= 6 mu= 0.0349233073728
(6,)
k_c= 6 mu= 0.281669260547
(6,)
k_c= 6 mu= 0.0342352903578
(6,)
k_c= 6 mu= 0.0570535107367
(6,)
k_c= 6 mu= 0.240365203795
(6,)
k_c= 6 mu= 0.46687933846
-0.379075247674
3 -th iteration is done
(6,)
k_c= 6 mu= 0.0732650710029
(6,)
k_c= 6 mu= 0.319929968688
(6,)
k_c= 6 mu= 0.0722511760734
(

In [212]:
F,W = AGKM(X,0.5,0.0001)

print 'F=',F, 'W=',W
#convert F, W into array type and calculate the difference
F = np.asarray(F)
W = np.asarray(W)
X = np.asarray(X)
print 'the difference is: ',np.sum(np.abs(F.dot(W)-X))

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+01  4e+00  2e+00  1e+00
 1: -5.9752e-01  1.4311e-02  4e-01  5e-01  3e-01  7e-01
 2: -6.0529e-02 -1.8725e-02  5e-02  4e-02  2e-02  5e-02
 3:  1.2234e-04  1.9521e-03  9e-03  6e-03  3e-03  3e-03
 4:  3.7915e-03  4.0765e-03  2e-03  1e-03  5e-04  5e-04
 5:  4.1547e-03  4.1850e-03  5e-04  3e-04  1e-04  1e-04
 6:  4.2869e-03  4.2880e-03  2e-05  1e-05  6e-06  4e-06
 7:  4.2940e-03  4.2940e-03  2e-07  1e-07  8e-08  5e-08
 8:  4.2941e-03  4.2941e-03  2e-09  1e-09  8e-10  5e-10
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+01  4e+00  2e+00  1e+00
 1: -3.9381e-01  2.3862e-01  1e+00  6e-01  4e-01  8e-01
 2:  5.9249e-04  1.0832e-01  2e-01  8e-02  5e-02  1e-01
 3:  1.6929e-01  2.1068e-01  1e-01  4e-02  2e-02  5e-02
 4:  1.9722e-01  2.0117e-01  1e-02  4e-03  2e-03  5e-03
 5:  2.0228e-01  2.0258e-01  2e-03  4e-04  3e-04  4e-04
 6:  2.0279e-01  2.0283e-01 